In [ ]:
import pennylane as qml
from pennylane import numpy as np

from tqdm import tqdm
from IPython.display import clear_output

import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from functools import reduce
import random
import math
import matplotlib.pyplot as plt
import pickle
import pandas as pd

## 금융 데이터를 yfinance로부터 받아오기

In [ ]:
import yfinance as yf
import datetime

def get_stocks_datas(ticker_names: list[str], start_date, end_date):
    # WARN: returns는 prices보다 길이가 1 작다.
    prices = {}
    returns = {}
    for name in ticker_names:
        data = yf.Ticker(name)
        data = data.history(start=start_date, end=end_date, interval="1d")
        prices[name] = data['Close']
        returns[name] = data['Close'].pct_change()[1:]
    return prices, returns

start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime(2017, 12, 31)
stock_prices, stock_returns = get_stocks_datas(["AAPL", "MSFT"], start_date, end_date)


In [ ]:
# 산점도 그리기
plt.scatter(stock_returns["AAPL"].values, stock_returns["MSFT"].values, alpha=0.5)
plt.title(f'AAPL/MSFT daily returns')
plt.xlabel('AAPL')
plt.ylabel('MSFT')
plt.xlim(-0.15, 0.15)
plt.ylim(-0.15, 0.15)
plt.grid(True)
plt.show()

## copula 변환/역변환 준비

역변환은 기존 분포를 normal distribution이라 가정

In [ ]:
from scipy.stats import norm
import random

def probability_integral_transform(x, y):
    assert len(x) == len(y)
    n = len(x)
    x_sorted = sorted([(x, i) for i, x in enumerate(x)])
    y_sorted = sorted([(y, i) for i, y in enumerate(y)])
    xx = np.zeros(n)
    yy = np.zeros(n)

    for i in tqdm(range(n)):
        xx[x_sorted[i][1]] = i / n
        yy[y_sorted[i][1]] = i / n
        
    return xx, yy

def reverse_probability_integral_transform(x, y, x_mean, x_std, y_mean, y_std):
    assert(len(x) == len(y))
    n = len(x)
    xx = []
    yy = []
    for i in range(n):
        # 정규분포에서 면적이 x[i] 가 되는 z값
        xx.append(norm.ppf(x[i], loc=x_mean, scale=x_std))
        yy.append(norm.ppf(y[i], loc=y_mean, scale=y_std))
    return xx, yy

In [ ]:
copula_x, copula_y = probability_integral_transform(stock_returns["AAPL"].values, stock_returns["MSFT"].values)
plt.scatter(copula_x, copula_y, s=20)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Copula space plot')

In [ ]:
rev_x, rev_y = reverse_probability_integral_transform(copula_x, copula_y,
                                                      stock_returns["AAPL"].mean(), stock_returns["AAPL"].std(),
                                                      stock_returns["MSFT"].mean(), stock_returns["MSFT"].std())
plt.scatter(rev_x, rev_y, alpha=0.5)
print(max(rev_x),max(rev_y))
plt.title(f'restored AAPL/MSFT daily returns')
plt.xlabel('AAPL')
plt.ylabel('MSFT')
plt.grid(True)
plt.show()

In [ ]:
print(len(stock_prices['AAPL'].values))
print(len(stock_prices['MSFT'].values))

In [ ]:
x = np.column_stack((stock_returns['AAPL'].values, stock_returns['MSFT'].values))
plt.scatter(x[:,0], x[:,1], s=2.0)
plt.title('Target distribution')
plt.show()

In [ ]:
# setting torch device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
noise_qubits = 6
code_qubits = 1
n_qubits = noise_qubits + code_qubits

n_layers = 1
BATCH_SIZE = 16
print("n_qubits = {} n_layers = {}".format(n_qubits, n_layers))

dev = qml.device("default.qubit", wires=n_qubits)

In [ ]:
coeff = 0

def generator_init(generator_input):
    for i in range(n_qubits):
        qml.RY(generator_input[i]*np.pi/2, wires=i) # TODO: *a 해서 값 범위 맞추기
    for i in range(noise_qubits//2):
        qml.Hadamard(wires=i)
    for i in range(noise_qubits//2):
        qml.CNOT(wires=[i, i+noise_qubits//2])

def generator_layer(params):
    for i in range(n_qubits):
        qml.RZ(params[i][0], wires=i)
        qml.RX(params[i][1], wires=i)
        qml.RZ(params[i][2], wires=i)
    
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1)%n_qubits])
        qml.RX(params[i][3], wires=i)
        qml.CNOT(wires=[i, (i+2)%n_qubits])

@qml.qnode(dev, interface="torch")
def generator_circuit(params, generator_input):
    """
    quantum circuit nodeq1
    generator_input (np.array(큐빗)) : 생성기 입력 seed (noise + code)
    params (torch.Tensor(레이어,큐빗,3)): a parameter
    마지막 측정은 모두 Z로
    """

    generator_init(generator_input)

    for param in params:
        generator_layer(param)

    return [qml.probs(wires=i) for i in range(n_qubits)]

def generator_forward(params, generator_input, copula=True):
    # 제너레이터 돌리고 결과 return하는 함수
    if copula:
        # copula space의 출력으로 변환 [0~1]범위
        generator_output = [generator_circuit(params, single_in)[::2] for single_in in generator_input]
        values = []
        for output in generator_output:
            x = 0
            y = 0
            for i in range(noise_qubits//2):
                x += (0.5**(i+1)) * output[i]
            for i in range(noise_qubits//2, noise_qubits):
                y += (0.5**(i-noise_qubits//2+1)) * output[i]
            x += random.randint(0, (2**20)-1) / (2**24)
            y += random.randint(0, (2**20)-1) / (2**24)
            values.append([x, y])
        generator_output = torch.tensor(values) # (batch_size, 2) 차원
        return generator_output
    else:
        # daily return 공간. 중심 0
        generator_output = [generator_circuit(params, single_in)[::2] for single_in in generator_input]
        generator_output = torch.stack(generator_output) # (BATCH_SIZE, n_qubits) 차원
        generator_output = (2 / np.pi * torch.arcsin(torch.sqrt(generator_output)))-0.5 # (BATCH_SIZE, n_qubits) 차원
        return generator_output[:, :2] # (BATCH_SIZE, 2) 차원

def generator_train_step(params, generator_input, use_mine = False, _qmine = False):
    '''
    params (torch.Tensor(레이어,큐빗,3)): a parameter
    generator_input (torch.Tensor(BATCH_SIZE, n_qubits)): 생성기 입력 seed (noise + code). -1~1 사이의 값
    '''
    code_input = generator_input[:, -code_qubits:] # 입력중에서 code만 뽑는다. (BATCH_SIZE, code_qubits)

    generator_output = generator_forward(params, generator_input, copula=output_copula) # 출력을 뽑아낸다
    generator_output = generator_output.to(torch.float32) # (BATCH_SIZE, output_qubits)
    
    disc_output = discriminator(generator_output) # 밑에 코드에서 정의됨
    gan_loss = torch.log(1-disc_output).mean()
    
    if use_mine:
        pred_xy = mine(code_input, generator_output)
        code_input_shuffle = code_input[torch.randperm(BATCH_SIZE)]
        pred_x_y = mine(code_input_shuffle, generator_output)
        mi = torch.mean(pred_xy) - torch.log(torch.mean(torch.exp(pred_x_y)))
        gan_loss -= coeff * mi

    elif _qmine:
        gan_loss += 0 # TODO: qmine loss 추가하기

    return generator_output, gan_loss

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class LinearDiscriminator(nn.Module):
    def __init__(self, input_dim=2):
        super().__init__()
        self.hidden_size = 100
        self.layers = nn.Sequential(
            nn.Linear(input_dim, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, 1),
            nn.Sigmoid())

        self.input_dim = input_dim

    def forward(self, x):
        if(len(x.shape) != 2):
            x = x.view(x.shape[0], -1)

        return self.layers(x)


class LinearMine(nn.Module):
    def __init__(self):
        super(LinearMine, self).__init__()
        H = 30
        self.fc1 = nn.Linear(code_qubits, H)
        self.fc2 = nn.Linear(2, H)
        self.fc3 = nn.Linear(H, 1)

    def forward(self, x, y):
        h1 = F.relu(self.fc1(x)+self.fc2(y))
        h2 = self.fc3(h1)
        return h2

disc_loss_fn = nn.BCELoss()
def disc_cost_fn(real_input, fake_input, smoothing=False):
    batch_num = real_input.shape[0]

    disc_real = discriminator(real_input)
    disc_fake = discriminator(fake_input)

    real_label = torch.ones((batch_num, 1)).to(device)
    fake_label = torch.zeros((batch_num, 1)).to(device)
    
    if smoothing:
        real_label = real_label - 0.2*torch.rand(real_label.shape).to(device)
    
    loss = 0.5 * (disc_loss_fn(disc_real, real_label) + disc_loss_fn(disc_fake, fake_label))
    
    return loss


In [ ]:
generator_params = Variable(torch.tensor(np.random.normal(0, np.pi, (n_layers, n_qubits, 4))), requires_grad=True)
print("parameter shape: ", generator_params.shape)

discriminator = LinearDiscriminator()
mine = LinearMine()

G_lr = 1e-3
D_lr = 1.5e-4
M_lr = 1e-3
output_copula = False # true면 [0~1]범위 출력. false면 대략 -0.1~0.1범위 daily returns
use_mine = True
use_qmine = False
G_opt = torch.optim.Adam([generator_params], lr=G_lr)
D_opt = torch.optim.Adam(discriminator.parameters(), lr=D_lr)
M_opt = torch.optim.Adam(mine.parameters(), lr=M_lr)

G_scheduler = torch.optim.lr_scheduler.StepLR(G_opt, step_size=30, gamma=0.7)
D_scheduler = torch.optim.lr_scheduler.StepLR(D_opt, step_size=30, gamma=0.85)
M_scheduler = torch.optim.lr_scheduler.StepLR(M_opt, step_size=30, gamma=0.7)

In [ ]:
import os
title = f'try5_returns_{use_mine}'
if not os.path.exists(f'result/{title}'):
    os.makedirs(f'result/{title}')
    
with open(f'result/{title}/param.txt', 'w') as f:
    f.write('G_lr = {}\n'.format(G_lr))
    f.write('D_lr = {}\n'.format(D_lr))
    f.write('M_lr = {}\n'.format(M_lr))
    f.write('G_scheduler: step={}, gamma={}\n'.format(G_scheduler.step_size, G_scheduler.gamma))
    f.write('D_scheduler: step={}, gamma={}\n'.format(D_scheduler.step_size, D_scheduler.gamma))
    f.write('M_scheduler: step={}, gamma={}\n'.format(M_scheduler.step_size, M_scheduler.gamma))
    f.write('coeff = {}\n'.format(coeff))
    f.write('output_copula = {}\n'.format(output_copula))
    f.write('use_mine = {}\n'.format(use_mine))
    f.write('use_qmine = {}\n'.format(use_qmine))
    f.write('n_qubits = {}\n'.format(n_qubits))
    f.write('noise_qubits = {}\n'.format(noise_qubits))
    f.write('code_qubits = {}\n'.format(code_qubits))
    f.write('n_layers = {}\n'.format(n_layers))
    f.write('param shape = {}\n'.format(generator_params.shape))

In [ ]:
def visualize_output(log_gen_outputs, log_gen_codes, title, rep, recorder):
    plt.figure(figsize=(10 + 4 * code_qubits, 7))  # 전체 그림의 크기 지정
    
    plt.subplot(2, 2 + code_qubits, 1)
    plt.title('Target distribution')
    plt.scatter(x[:,0], x[:,1], s=10,  alpha=0.2)
    plt.xlim((-0.2, 0.2))
    plt.ylim((-0.2, 0.2))
    plt.grid()

    plt.subplot(2, 2 + code_qubits, 2)
    plt.title('Epoch {0}'.format(rep))
    plt.scatter(log_gen_outputs[:,0], log_gen_outputs[:,1], s=10,  alpha=0.2)
    plt.xlim((-0.2, 0.2))
    plt.ylim((-0.2, 0.2))
    plt.grid()

    for i in range(code_qubits):
        plt.subplot(2, 2 + code_qubits, 3 + i)
        plt.title('Epoch {0} code {1}'.format(rep, i))
        plt.scatter(log_gen_outputs[:,0], log_gen_outputs[:,1], s=10, c=log_gen_codes[:, i], cmap='RdYlBu', alpha=0.2)
        plt.xlim((-0.2, 0.2))
        plt.ylim((-0.2, 0.2))
        plt.colorbar()  # 색상 막대 추가
        plt.grid()
    
    gen_copula_x, gen_copula_y = probability_integral_transform(log_gen_outputs[:,0], log_gen_outputs[:,1])
    
    plt.subplot(2, 2 + code_qubits, 3 + code_qubits)
    plt.title('Target copula'.format(rep))
    plt.scatter(copula_x, copula_y, s=10,  alpha=0.2)
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    plt.grid()

    plt.subplot(2, 2 + code_qubits, 4 + code_qubits)
    plt.title('Epoch {0} copula'.format(rep))
    plt.scatter(gen_copula_x, gen_copula_y, s=10,  alpha=0.2)
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    plt.grid()

    for i in range(code_qubits):
        plt.subplot(2, 2 + code_qubits, 5 + code_qubits + i)
        plt.title('Epoch {0} code {1}'.format(rep, i))
        plt.scatter(gen_copula_x, gen_copula_y, s=10, c=log_gen_codes[:, i], cmap='RdYlBu', alpha=0.2)
        plt.xlim((0, 1))
        plt.ylim((0, 1))
        plt.colorbar()  # 색상 막대 추가
        plt.grid()

    plt.savefig(f'result/{title}/{rep}.png', dpi=300)
    plt.show()
    
    plt.figure(figsize=(10, 4))
    plt.title('Epoch {0} code-axis corr'.format(rep))
    for i in range(code_qubits):
        plt.plot(recorder[f'code{i}-x'], label=f'code{i}-x', marker='o')
        plt.plot(recorder[f'code{i}-y'], label=f'code{i}-y', marker='o')
        plt.plot(recorder[f'mi'], label='mi', marker='o')
        plt.plot(recorder[f'd_loss'], label='d_loss', marker='o')
        plt.plot(recorder[f'g_loss'], label='g_loss', marker='o')

    plt.xlabel('epoch')
    plt.ylabel('correlation')
    plt.ylim(-1, 1)
    plt.title(f'code - axis corr graph (rep={rep})')
    plt.legend()
    plt.savefig(f'result/{title}/corr_{rep}.png', dpi=300)
    plt.show()
    

In [ ]:
epoch = 300

recorder_keywords = ['d_loss', 'g_loss', 'mi', 'corr']
for i in range(code_qubits):
    recorder_keywords.append(f'code{i}-x')
    recorder_keywords.append(f'code{i}-y')

recorder = {k: [] for k in recorder_keywords}
final_rep = 0

c_target = np.corrcoef(x[:,0], x[:,1])[0,1]

for rep in range(1, epoch+1):
    np.random.shuffle(x)
    iter_num = int(len(x) //BATCH_SIZE) # 매번 50% 추출해서 학습. 셔플하니까 자투리 생기는건 무시.
    
    G_loss_sum = 0.0
    D_loss_sum = 0.0
    a_loss_sum = 0.0
    b_loss_sum = 0.0
    mi_sum = 0.0
    pbar = tqdm(range(iter_num))
    log_gen_outputs = []
    log_gen_codes = []

    coeff = (1 + math.tanh((rep-100)/50))/10 # epoch100부터 mine을 고려하기 시작함
    
    for i in pbar:
        batch = torch.FloatTensor(x[BATCH_SIZE * i : BATCH_SIZE * i + BATCH_SIZE])
        
        # train generator
        generator_seed = torch.rand((BATCH_SIZE, n_qubits)) * 2 - 1
        generator_output, generator_loss = generator_train_step(generator_params, generator_seed, use_mine=use_mine, _qmine=use_qmine)
        G_opt.zero_grad()
        generator_loss.requires_grad_(True)
        generator_loss.backward()
        G_opt.step()

        # train discriminator
        fake_input = generator_output.detach().to(torch.float32)
        disc_loss = disc_cost_fn(batch, fake_input, smoothing=False)
        D_opt.zero_grad()
        disc_loss.requires_grad_(True)
        disc_loss.backward()
        D_opt.step()

        # train mine
        code_input = generator_seed[:, -code_qubits:] 
        pred_xy = mine(code_input, fake_input)
        code_input_shuffle = code_input[torch.randperm(BATCH_SIZE)]
        pred_x_y = mine(code_input_shuffle, fake_input)
        mi = -torch.mean(pred_xy) + torch.log(torch.mean(torch.exp(pred_x_y)))
        M_opt.zero_grad()
        mi.requires_grad_(True)
        mi.backward()
        M_opt.step()


        D_loss_sum += disc_loss.item()
        G_loss_sum += generator_loss.item()
        mi_sum -= mi.item() # (-1)곱해져 있어서 빼야함.

        pbar.set_postfix({'G_loss': G_loss_sum/(i+1), 'D_loss': D_loss_sum/(i+1), 'MI': mi_sum/(i+1)})
        log_gen_outputs.append(fake_input.numpy())
        log_gen_codes.append(code_input.numpy())

    G_scheduler.step()
    D_scheduler.step()
    M_scheduler.step()

    recorder['d_loss'].append(D_loss_sum/iter_num)
    recorder['g_loss'].append(G_loss_sum/iter_num)
    recorder['mi'].append(mi_sum/iter_num)
    
    log_gen_outputs = np.concatenate(log_gen_outputs, axis=0)
    log_gen_codes = np.concatenate(log_gen_codes, axis=0)
    print("epoch: {}, D_loss: {}, G_loss: {}, MI = {}".format(rep, D_loss_sum/iter_num, G_loss_sum/iter_num, mi_sum/iter_num))
    print("좌표값 평균 = ", np.mean(log_gen_outputs[:,0]), np.mean(log_gen_outputs[:,1]))
    c_gen = np.corrcoef(log_gen_outputs[:,0], log_gen_outputs[:,1])[0,1]
    
    print("corr = ", "진짜 corr = ",c_gen, c_target)
    recorder['corr'].append(c_gen)
    
    df = pd.DataFrame({'x': log_gen_outputs[:, 0], 'y': log_gen_outputs[:, 1]})
    for i in range(code_qubits):
        df[f'code{i}']=log_gen_codes[:, i]
    corr_mat = df.corr().to_numpy()
    for i in range(code_qubits):
        recorder[f'code{i}-x'].append(corr_mat[0, i+2])
        recorder[f'code{i}-y'].append(corr_mat[1, i+2])
    
    visualize_output(log_gen_outputs, log_gen_codes, title, rep, recorder)

    if rep % 10 == 0:
        with open(f'result/{title}/discriminator_{rep}.pkl', 'wb') as file:
            pickle.dump(discriminator, file)
        with open(f'result/{title}/generator_{rep}.pkl', 'wb') as file:
            pickle.dump(generator_params, file)

    final_rep = rep

## 모든 지표 엑셀파일로 저장

In [ ]:
import openpyxl
df = pd.DataFrame(recorder)
output_filename = f'result/{title}/recorder.xlsx'
df.to_excel(output_filename, index=False)

## 최종 결과 plot

In [ ]:
inputs = []
outputs = []

for i in tqdm(range(5000)):
    with torch.no_grad():
        z = np.random.uniform(-1, 1, (1, n_qubits, 1))
        code_input = z[:, -code_qubits:].reshape(code_qubits) # 입력 z중에서 code를 추출한다.
        generator_output = generator_forward(generator_params, z, copula=output_copula)
        generator_output = generator_output.cpu().numpy().reshape(2)
        outputs.append(generator_output)
        inputs.append(code_input)

inputs = np.array(inputs).reshape(-1, code_qubits)

for code_ind in range(code_qubits):
    outputs = np.array(outputs)
    plt.scatter(outputs[:, 0], outputs[:, 1], c=inputs[:, code_ind], cmap='RdYlBu', alpha=0.2)
    plt.colorbar()  # 색상 막대 추가
    plt.title(f'code{code_ind}-distribution (rep = {final_rep})')
    plt.savefig(f'result/{title}/code_{code_ind}_{final_rep}.png')
    plt.show()

In [ ]:
outputs = np.array(outputs)[:len(x)]
plt.scatter(outputs[:, 0], outputs[:, 1], s=17, alpha=0.1)
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)
plt.show()

In [ ]:
pd.DataFrame({'x': outputs[:,0], 'y': outputs[:,1]}).corr()

In [ ]:
outputs = np.array(outputs)
plt.scatter(x[:, 0], x[:, 1], s=17, alpha=0.1)
plt.xlim(-0.1, 0.1)
plt.ylim(-0.1, 0.1)
plt.show()
pd.DataFrame({'x': x[:,0], 'y': x[:,1]}).corr()